In [6]:
from dataclasses import dataclass, field
from marshmallow import Schema, fields, post_load, post_dump

In [7]:
from uuid import uuid4

def make_uuid(prefix):
    def _make_uuid():
        return '-'.join([prefix, str(uuid4())])
    return _make_uuid

In [8]:
@dataclass(order=True)
class Todo:
    user_id: int = field()
    _id: int = field(compare=False, repr=False)
    uuid: str = field()
    title: str = field()
    completed: bool = field(compare=False)
        

In [9]:
class TodoSchema(Schema):
    user_id = fields.Integer()
    _id = fields.Integer(missing=-1)
    uuid = fields.String(missing=make_uuid('TODO'))
    title = fields.String()
    completed = fields.Boolean(missing=False)
    
    @post_load
    def make_todo(self, data, **kwargs):
        return Todo(**data)
    
    @post_dump
    def maybe_remove_id(self, data, **kwargs):
        if data['_id']==-1:
            data.pop('_id')
        return data

In [10]:
t1 = TodoSchema().load({
    'user_id': 1,
    'title': 'Lorem ipsum',
})

t2 = TodoSchema().load({
    'user_id': 0,
    'title': 'Lorem ipsum',
    'completed': True
})

t3 = TodoSchema().load({
    'user_id': 0,
    'title': 'Lorem ipsum dolor sit amet'
})

In [11]:
dumped  = TodoSchema().dump(t1)

In [12]:
dumped

{'uuid': 'TODO-8b1fb879-7f17-4d0a-8cfb-73b447bc917a',
 'title': 'Lorem ipsum',
 'user_id': 1,
 'completed': False}

In [13]:
t1==t2

False

In [14]:
todos = [t1,t2,t3]
todos.sort()
todos

[Todo(user_id=0, uuid='TODO-6b6eb59f-6302-4bee-a4cc-f745f285deb1', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-ac0f557a-f464-4a9f-85e5-8e8f8cb542c4', title='Lorem ipsum', completed=True),
 Todo(user_id=1, uuid='TODO-8b1fb879-7f17-4d0a-8cfb-73b447bc917a', title='Lorem ipsum', completed=False)]

In [15]:
todos_list = TodoSchema(many=True).dump(todos)
todos_list

[{'uuid': 'TODO-6b6eb59f-6302-4bee-a4cc-f745f285deb1',
  'title': 'Lorem ipsum dolor sit amet',
  'user_id': 0,
  'completed': False},
 {'uuid': 'TODO-ac0f557a-f464-4a9f-85e5-8e8f8cb542c4',
  'title': 'Lorem ipsum',
  'user_id': 0,
  'completed': True},
 {'uuid': 'TODO-8b1fb879-7f17-4d0a-8cfb-73b447bc917a',
  'title': 'Lorem ipsum',
  'user_id': 1,
  'completed': False}]

In [16]:
TodoSchema(many=True).load(todos_list)

[Todo(user_id=0, uuid='TODO-6b6eb59f-6302-4bee-a4cc-f745f285deb1', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-ac0f557a-f464-4a9f-85e5-8e8f8cb542c4', title='Lorem ipsum', completed=True),
 Todo(user_id=1, uuid='TODO-8b1fb879-7f17-4d0a-8cfb-73b447bc917a', title='Lorem ipsum', completed=False)]

In [17]:
import json

class MockDao:
    
    def __init__(self, path, schema):
        self._path = path
        self._schema = schema
        try:
            with open(self._path,'r') as f:
                self._items=self._schema.load(json.load(f))
        except:
            self._items = []
    
    def get_all(self):
        return self._items
    
    def get_by_uuid(self, uuid):
        item = [t for t in self._items if t.uuid==uuid]
        return item[0] if item else None
    
    def add_item(self, item):
        self._items.append(item)
        self._replace_file()
        
    def delete_by_uuid(self, uuid):
        item = [t for t in self._items if t.uuid==uuid]
        if item:
            self._items.pop(self._items.index(item[0]))
        self._replace_file()
            
    def update_item_by_uuid(self, uuid, data):
        item = self.get_by_uuid(uuid)
        for k,v in data.items():
            setattr(item,k,v)
        self._replace_file()
        
    def _replace_file(self):
        with open(self._path, 'w') as f:
            serialized = self._schema.dump(self._items)
            json.dump(serialized,f)
        
    def clear(self):
        self._items = []
        self._replace_file()

In [26]:
TODOS_DB = "/todos/database/todos_db.json"
dao = MockDao(TODOS_DB, TodoSchema(many=True))
# dao.clear()

In [29]:
dao.get_all()

[Todo(user_id=0, uuid='TODO-6b6eb59f-6302-4bee-a4cc-f745f285deb1', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-ac0f557a-f464-4a9f-85e5-8e8f8cb542c4', title='Lorem ipsum', completed=True),
 Todo(user_id=1, uuid='TODO-8b1fb879-7f17-4d0a-8cfb-73b447bc917a', title='Lorem ipsum', completed=False)]

In [28]:
schema = TodoSchema(many=True)
copied_todos = schema.load(schema.dump(todos))
for t in copied_todos:
    dao.add_item(t) 

In [41]:
dao._items[2] is todos[2]

False

In [42]:
dao._items[2] == todos[2]

True

In [43]:
dao.get_all()

[Todo(user_id=0, uuid='TODO-33511623-37d8-49e4-96cf-bb0abdc3ef8e', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-5b4542ea-1b5f-4936-bee3-feeb3d178666', title='Lorem ipsum', completed=True),
 Todo(user_id=1, uuid='TODO-d7af4c5f-bd21-4bfc-8368-451a130a078c', title='Lorem ipsum', completed=False)]

In [30]:
dao.get_by_uuid(todos[2].uuid)

Todo(user_id=1, uuid='TODO-8b1fb879-7f17-4d0a-8cfb-73b447bc917a', title='Lorem ipsum', completed=False)

In [45]:
dao.delete_by_uuid(todos[0].uuid)

In [46]:
dao.get_all()

[Todo(user_id=0, uuid='TODO-5b4542ea-1b5f-4936-bee3-feeb3d178666', title='Lorem ipsum', completed=True),
 Todo(user_id=1, uuid='TODO-d7af4c5f-bd21-4bfc-8368-451a130a078c', title='Lorem ipsum', completed=False)]

In [47]:
dao.update_item_by_uuid(todos[2].uuid, {'user_id':0})

In [225]:
dao.get_all()

[Todo(user_id=0, uuid='TODO-ed8f3d80-9a62-4326-ac7b-c5411e7e8113', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-43ddbf6e-d5ca-4a8c-8f61-033ea59f3d50', title='Lorem ipsum', completed=False)]

In [163]:
todos

[Todo(user_id=0, uuid='TODO-56b06af7-45e3-413f-84bc-895abb99bdb3', title='Lorem ipsum', completed=True),
 Todo(user_id=0, uuid='TODO-ed8f3d80-9a62-4326-ac7b-c5411e7e8113', title='Lorem ipsum dolor sit amet', completed=False),
 Todo(user_id=0, uuid='TODO-43ddbf6e-d5ca-4a8c-8f61-033ea59f3d50', title='Lorem ipsum', completed=False)]